In [24]:
import simpy 
import random
import statistics
import numpy

In [76]:
import simpy 
import random
import statistics
import numpy

wait_times = []

class Airport(object):
    def __init__(self, env, num_workers, num_queues):
        self.env = env
        self.worker = simpy.Resource(env, num_workers)
        self.queue = simpy.Resource(env, num_queues)
        
    def id_check(self, passenger):
        yield self.env.timeout(random.expovariate(1.33))    
        
    def personal_check(self, passenger):
        yield self.env.timeout(numpy.random.uniform(0.5, 1.0))
        
def go_to_airport(env, passenger, airport):
    arrival_time = env.now

    with airport.worker.request() as request:
        yield request
        yield env.process(airport.id_check(passenger))

    with airport.queue.request() as request:
        yield request
        yield env.process(airport.personal_check(passenger))
    
    wait_times.append(env.now - arrival_time)

def run_airport(env, num_workers, num_queues):
    airport = Airport(env, num_workers, num_queues)
    
    for passenger in range(1):
        env.process(go_to_airport(env, passenger, airport))

    while True:
        yield env.timeout(random.expovariate(50))
        passenger += 1
        env.process(go_to_airport(env, passenger, airport))

def avg_wait_time(wait_times):
    average_wait = statistics.mean(wait_times)
    return average_wait

def get_user_input():
    num_workers = input("Input # of workers: ")
    num_queues = input("Input # of queues: ")

    params = [num_workers, num_queues]
    if all(str(i).isdigit() for i in params):  # Check input is valid
        params = [int(x) for x in params]
    else:
        print(
            "Could not parse input. The simulation will use default values:",
            "\n1 cashier, 1 server, 1 usher.",
        )
        params = [1, 1]
    return params

def main():
    # Setup
    random.seed(42)
    num_workers, num_queues = get_user_input()
    
    # Run the simulation
    env = simpy.Environment()
    env.process(run_airport(env, num_workers, num_queues))
    env.run(until=120)

    # View the results
    mins = avg_wait_time(wait_times)
    print(
      "Running simulation...",
      f"\nThe average wait time is {mins} minutes.",
    )
        
main()
        

Input # of workers: 50
Input # of queues: 20
Running simulation... 
The average wait time is 28.163146416658165 minutes.


In [74]:
#random.expovariate(1.33)
random.expovariate(.75)

0.7551111328258333

In [ ]:
#### def clock(env, name, tick):
    while True:
        print(name, env.now)
        yield env.timeout(tick)

In [4]:
env = simpy.Environment()
env.process(clock(env, 'fast', 0.5))
env.process(clock(env, 'slow', 1))
env.run(until=2)

fast 0
slow 0
fast 0.5
slow 1
fast 1.0
fast 1.5
